# Transformation de CV en Dossier de Compétences

Ce notebook présente un pipeline complet pour transformer un CV en dossier de compétences.

Les étapes comprennent :

- **Extraction de texte** : Soit via une extraction native (PDF sélectionnable) ou par OCR (PaddleOCR).
- **Extraction OCR-Free** : Utilisation du modèle Donut pour extraire des informations directement depuis l’image.
- **Conversion JSON** : Transformation de la sortie JSON en un format lisible par l’humain.
- **Calcul de similarité** : Comparaison des compétences extraites avec un ensemble idéal.
- **Extraction NER** : Utilisation de Camembert pour la reconnaissance d’entités (extraction de compétences, langues, etc.).


In [1]:
# Imports nécessaires
import fitz  # PyMuPDF
import pytesseract
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
from langdetect import detect
from paddleocr import PaddleOCR
import json

# Pour l'approche OCR-Free avec Donut
from transformers import DonutProcessor, VisionEncoderDecoderModel

# Pour NER avec Camembert
from transformers import pipeline, CamembertTokenizer

# Configurez le chemin de Tesseract (à adapter selon votre environnement)
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\mathi\anaconda3\Lib\site-packages\pytesseract\tesseract.exe"

ModuleNotFoundError: No module named 'fitz'

## 1. Détection de la langue d'une image

Cette fonction utilise Tesseract pour extraire le texte depuis une image, puis `langdetect` pour déterminer la langue.

**Exemple toy :** Nous créons une image simple contenant du texte.

In [ ]:
def detect_language_from_image(image):
    # Extraction de texte avec Tesseract (rapide)
    text = pytesseract.image_to_string(image, lang="eng+fra")
    
    if text.strip():
        try:
            detected_lang = detect(text)
            print(f"🔍 Detected Language: {detected_lang}")  # Debug
            if detected_lang.startswith("fr"):
                return "fr"
            elif detected_lang.startswith("en"):
                return "en"
        except Exception as e:
            print(f"Erreur de détection de langue: {e}")
            return "unknown"
    return "unknown"

**Toy Example 1 – Détection de la langue :**

Créons une image contenant du texte en français.

In [ ]:
# Création d'une image blanche avec du texte
toy_img = Image.new('RGB', (300, 100), color=(255, 255, 255))
draw = ImageDraw.Draw(toy_img)
# Utilisation d'une police par défaut (si disponible)
try:
    font = ImageFont.truetype("arial.ttf", 20)
except IOError:
    font = None
draw.text((10, 40), "Bonjour, comment ça va ?", fill=(0, 0, 0), font=font)

lang_toy = detect_language_from_image(toy_img)
print("Langue détectée (toy):", lang_toy)

## 2. Extraction de texte via OCR avec PaddleOCR

Cette fonction extrait le texte d'un PDF en utilisant PaddleOCR.

**Remarque :** Pour cet exemple, un PDF réel est nécessaire. Ici, nous présentons la fonction.

$$\text{Exemple : } \text{extract\_text\_with\_paddleocr(pdf\_path)}$$

In [ ]:
def extract_text_with_paddleocr(pdf_path):
    extracted_text = ""
    with fitz.open(pdf_path) as pdf:
        if len(pdf) == 0:
            print("Erreur: Le PDF est vide ou corrompu.")
            return ""
        
        for page_num, page in enumerate(pdf, start=1):
            zoom = 2.0  # Augmentation de la résolution pour l'OCR
            mat = fitz.Matrix(zoom, zoom)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Détection de la langue de la page
            lang = detect_language_from_image(img)
            
            # Sélection du modèle PaddleOCR en fonction de la langue détectée
            if lang == "fr":
                print("🇫🇷 Utilisation de PaddleOCR en mode français...")
                ocr = PaddleOCR(use_angle_cls=True, lang="latin", use_gpu=False)
            else:
                print("🇬🇧 Utilisation de PaddleOCR en mode anglais...")
                ocr = PaddleOCR(use_angle_cls=True, lang="en", use_gpu=False)
            
            # Exécution de l'OCR avec PaddleOCR
            ocr_result = ocr.ocr(np.array(img), det=True, rec=True, cls=True)
            extracted_text += f"\n--- Page {page_num} (OCR) ---\n"
            
            if ocr_result and isinstance(ocr_result, list) and ocr_result[0]:
                for line in ocr_result[0]:
                    if isinstance(line, list) and len(line) > 1:
                        text_tuple = line[1]
                        if isinstance(text_tuple, tuple) and len(text_tuple) > 0:
                            text_value = text_tuple[0]
                            if isinstance(text_value, str):
                                extracted_text += text_value + "\n"
            else:
                extracted_text += "\n[Pas de texte détecté sur cette page]\n"
    return extracted_text

## 3. Pipeline d'extraction du PDF

Cette fonction détermine si l'extraction native du texte depuis le PDF (via PyMuPDF) est possible.
Si le texte extrait est insuffisant (moins de 100 caractères), elle utilise l'OCR avec PaddleOCR.

$$\text{Si } |text| > 100 \text{ alors extraction native, sinon utilisation de l'OCR.}$$

In [ ]:
def extraction_pipeline(pdf_path):
    # Extraction native via PyMuPDF
    text_native = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text_native += page.get_text()
    
    if len(text_native.strip()) > 100:
        print("Extraction native réussie, utilisation du texte extrait.")
        return text_native
    else:
        print("Extraction native insuffisante, utilisation de l'OCR.")
        return extract_text_with_paddleocr(pdf_path)

## 4. Extraction OCR-Free avec Donut

Utilisation du modèle Donut pour extraire des informations directement depuis l'image, sans passer par un OCR explicite.

**Étapes :**
1. Charger le modèle et le processor.
2. Convertir la première page du PDF en image.
3. Extraire le contenu via Donut.

$$\text{Donut output : une chaîne JSON structurée.}$$

In [ ]:
# Chargement du modèle Donut
donut_model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base")
donut_processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")

def extract_with_donut(pdf_path):
    # Traitement de la première page du PDF
    with fitz.open(pdf_path) as pdf:
        page = pdf[0]
        zoom = 2.0
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat, alpha=False)
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    pixel_values = donut_processor(image, return_tensors="pt").pixel_values
    outputs = donut_model.generate(pixel_values)
    decoded_output = donut_processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output

## 5. Conversion du JSON en format lisible

Cette fonction prend en entrée une chaîne JSON (par exemple, la sortie du modèle Donut) et la convertit en un texte formaté pour une lecture humaine.

$$\text{Conversion : JSON } \to \text{ texte lisible.}$$

In [ ]:
def json_to_human_readable(json_output):
    try:
        data = json.loads(json_output)
    except json.JSONDecodeError:
        print("Erreur: La sortie n'est pas un JSON valide.")
        return json_output  # Retourne la chaîne brute si le JSON n'est pas valide
    
    # Formatage du JSON de manière lisible
    readable = ""
    # On suppose que le JSON a une structure type dossier de compétences.
    if "skills_portfolio" in data:
        portfolio = data["skills_portfolio"]
        if "personal_info" in portfolio:
            info = portfolio["personal_info"]
            readable += f"Nom: {info.get('name', 'N/A')}\n"
            readable += f"Résidence: {info.get('residence', 'N/A')}\n\n"
        if "professional_experience" in portfolio:
            readable += "Expérience professionnelle:\n"
            for exp in portfolio["professional_experience"]:
                readable += f"- Entreprise: {exp.get('company', 'N/A')}, Durée: {exp.get('duration', 'N/A')}\n"
                readable += f"  Contexte: {exp.get('context', '')}\n"
                readable += f"  Réalisations: {exp.get('achievements', '')}\n\n"
        if "education" in portfolio:
            readable += "Formation:\n"
            for edu in portfolio["education"]:
                readable += f"- {edu.get('degree', 'N/A')} ({edu.get('period', 'N/A')}) à {edu.get('school', 'N/A')}\n"
            readable += "\n"
        if "languages" in portfolio:
            readable += "Langues:\n"
            for lang in portfolio["languages"]:
                readable += f"- {lang.get('name', 'N/A')} ({lang.get('level', 'N/A')})\n"
            readable += "\n"
        if "skills" in portfolio:
            readable += "Compétences techniques:\n"
            readable += ", ".join(portfolio["skills"]) + "\n\n"
        if "similarity_score" in portfolio:
            readable += f"Score de similarité: {portfolio['similarity_score']}\n"
    else:
        # Sinon, on retourne le JSON formaté avec indentations
        readable = json.dumps(data, indent=4, ensure_ascii=False)
    return readable

## 6. Calcul de Similarité entre compétences

La fonction ci-dessous calcule le score de similarité entre les compétences d'un candidat et un ensemble idéal.

L'équation utilisée est :  
$$\text{Score} = \frac{n}{N} \times 100$$  

où $n$ est le nombre de compétences communes et $N$ le nombre total de compétences idéales.

**Toy Example :**

```python
score = compute_similarity({"Python", "R"}, {"Python", "R", "SQL"})
# Résultat attendu : environ 66.67%
```

In [ ]:
def compute_similarity(candidate_skills, ideal_skills):
    common_skills = candidate_skills.intersection(ideal_skills)
    score = (len(common_skills) / len(ideal_skills)) * 100
    return round(score, 2)

## 7. Utilisation de Camembert pour la NER

Cette section montre comment utiliser le modèle Camembert NER pour extraire des entités du texte.

Nous définissons d'abord quelques listes prédéfinies de compétences, langues, soft skills, diplômes, secteurs et notions mathématiques.

Ensuite, la fonction `extract_ner` applique le modèle Camembert sur le texte (après normalisation avec `.title()`)
et catégorise chaque entité en fonction des listes.

$$\text{Camembert NER : } \text{ Texte } \to \text{ Entités catégorisées}$$

In [ ]:
# Listes prédéfinies
competences = {
    "Python", "R", "Java", "C++", "SQL", "JavaScript", "Docker", "Kubernetes", "Excel", "Power BI", "Git",
    "Tensorflow", "Pytorch", "Scikit-Learn", "Numpy", "Scipy", "Matplotlib", "Pandas", "AWS", "Linux", "TCP/IP",
    "PostgreSQL", "Scrum", "JIRA", "CI/CD", "Machine Learning", "Deep Learning", "Reinforcement Learning", "NLP", "Computer Vision",
    "MySQL", "MongoDB", "NoSQL", "Spark", "Hadoop"
}

langues = { "Français", "Anglais", "Espagnol", "Allemand", "Chinois", "Russe", "Arabe", "Italien", "Portugais"}

soft_skills = {
    "Leadership", "Communication", "Esprit D'équipe", "Adaptabilité", "Créativité",
    "Autonomie", "Résolution De Problèmes", "Gestion Du Temps", "Esprit Analytique", 
    "Gestion Du Stress", "Organisation", "Initiative", "Planification", "Proactivité",
    "Gestion De Projet", "Coordination", "Collaboration", "Gestion De Conflit", "Vulgarisation",
    "Présentation Orale", "Discipline", "Empathie", "Tolérance", "Confiance En Soi", "Persévérance",
    "Curiosité", "Polyvalence", "Mémoire", "Esprit Critique"
}

diplomes_certif = {
    "BTS", "DUT", "Licence", "Master", "Doctorat", "PhD", "MBA", 
    "TOEIC", "TOEFL", "IELTS", "Certifié AWS", "PMP", "Scrum Master", 
    "Cisco CCNA", "Microsoft Azure", "Google Cloud Professional", 
    "Coursera Deep Learning", "MITx Machine Learning"
}

secteurs = {
    "Finance", "Banque", "Assurance", "Santé", "Pharmaceutique", "Biotechnologies", "Aéronautique", "Aérospatial", "Défense",  
    "Automobile", "Télécommunications", "Énergie", "Nucléaire", "Énergies Renouvelables", "Transport", "Ferroviaire", "Maritime",  
    "E-commerce", "Grande Distribution", "Éducation", "Cybersécurité", "Automatisation", "Robotique", "Informatique", "Cloud Computing", 
    "Big Data", "Intelligence Artificielle", "Domotique", "Agroalimentaire", "Environnement",  
    "Luxe", "Mode", "Médias", "Jeux Vidéo"
}

maths_algo = {
    "Statistiques", "Probabilités", "Test D'Hypothèses", "Estimation", "Régression", "Séries Temporelles",  
    "Optimisation", "Programmation Linéaire", "Optimisation Combinatoire", "Programmation Dynamique", "Optimisation Convexe",
    "Descente De Gradient", "Algorithmes D'Optimisation Stochastique", "Graphes", "Réseaux", "Théorie Des Graphes",  
    "Algorithmes De Tri", "Algorithmes Sur Les Graphes", "Programmation Parallèle", "Analyse Numérique", "Équations Différentielles", 
    "Approximation Numérique", "Interpolation", "Monte-Carlo", "Intégration Numérique", "Transformée De Fourier", "Processus Markoviens",  
    "Algorithmes EM", "Traitement D'Images", "Analyse Topologique Des Données", "Cryptographie", "Théorie Des Nombres"
}

def extract_ner(text):
    # Charger le modèle Camembert NER
    tokenizer = CamembertTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
    ner = pipeline("ner", model="Jean-Baptiste/camembert-ner", tokenizer=tokenizer, grouped_entities=True)
    
    # Normalisation du texte
    text_capitalized = text.title()
    
    results = ner(text_capitalized)
    
    # Initialiser les listes pour chaque catégorie
    comp = []
    lang_list = []
    soft = []
    dipl = []
    sect = []
    math = []
    
    # Parcourir les résultats et catégoriser les entités
    for entity in results:
        word = entity["word"]
        entity_type = entity["entity_group"]
        score = entity["score"]
        
        if word in competences:
            entity_type = "COMPETENCE"
            comp.append(word)
        if word in langues:
            entity_type = "LANGUE"
            lang_list.append(word)
        if word in soft_skills:
            entity_type = "SOFT SKILL"
            soft.append(word)
        if word in diplomes_certif:
            entity_type = "DIPLOME"
            dipl.append(word)
        if word in secteurs:
            entity_type = "SECTEUR"
            sect.append(word)
        if word in maths_algo:
            entity_type = "MATHS/ALGO"
            math.append(word)
            
        print(f"{word} -> {entity_type} ({score:.2f})")
    
    return {
        "competences": comp,
        "langues": lang_list,
        "soft_skills": soft,
        "diplomes_certif": dipl,
        "secteurs": sect,
        "maths_algo": math
    }

**Toy Example 2 – Extraction NER avec Camembert :**

Nous utilisons un petit texte toy qui contient des compétences connues.

In [ ]:
toy_text = "Python and Machine Learning are key skills. I also know R and SQL."
ner_toy = extract_ner(toy_text)
print("Résultats NER (toy):")
print(ner_toy)

## 8. Tests et Exemples complémentaires

Voici quelques exemples supplémentaires pour illustrer les fonctions qui ne nécessitent pas de fichiers externes.

**Toy Example – Calcul de similarité :**

```python
score = compute_similarity({"Python", "R"}, {"Python", "R", "SQL"})
# Résultat attendu : environ 66.67%
```

In [ ]:
toy_candidate_skills = {"Python", "R"}
toy_ideal_skills = {"Python", "R", "SQL"}
similarity_toy = compute_similarity(toy_candidate_skills, toy_ideal_skills)
print("Score de similarité (toy) :", similarity_toy, "%")

**Toy Example – Conversion JSON en format lisible :**

Nous utilisons un JSON toy représentant un dossier de compétences.

In [ ]:
toy_json = """
{
    "skills_portfolio": {
        "personal_info": {
            "name": "Alice Dupont",
            "residence": "Lyon, France"
        },
        "professional_experience": [
            {
                "duration": "2020-2022",
                "company": "Startup X",
                "context": "Développement d'applications Python",
                "achievements": "Création d'un prototype innovant"
            }
        ],
        "education": [
            {
                "period": "2015-2018",
                "degree": "Licence en Informatique",
                "school": "Université de Lyon"
            }
        ],
        "languages": [
            {"name": "French", "level": "Natif"}
        ],
        "skills": ["Python", "Machine Learning"],
        "similarity_score": "75%"
    }
}
"""
readable_toy = json_to_human_readable(toy_json)
print("Conversion JSON (toy) -> Lisible:")
print(readable_toy)

## 9. Fonction Main – Pipeline complet

La fonction principale intègre l'ensemble du pipeline et affiche les résultats.

**Remarque :** Pour les fonctions d'extraction de PDF (extraction_pipeline et extract_with_donut),
un chemin vers un PDF réel est nécessaire. Les exemples toy présentés ci-dessus montrent les autres parties du pipeline.

In [ ]:
def main():
    # Chemin vers un PDF réel (à adapter)
    pdf_path = "C:\\Users\\mathi\\Desktop\\Mathis_Chabaud_CV.pdf"
    
    print("=== Début du pipeline d'extraction ===")
    
    # Extraction via pipeline (texte natif ou OCR)
    text_output = extraction_pipeline(pdf_path)
    print("=== Texte extrait ===")
    print(text_output)
    
    # Extraction OCR-Free avec Donut
    donut_result = extract_with_donut(pdf_path)
    print("=== Donut Output ===")
    print(donut_result)
    
    # Conversion de la sortie JSON en format lisible
    print("=== Conversion JSON -> Lisible ===")
    readable = json_to_human_readable(donut_result)
    print(readable)
    
    # Extraction NER via Camembert sur le texte extrait
    print("=== Extraction NER via Camembert ===")
    ner_res = extract_ner(text_output)
    print("Entités extraites:")
    print(ner_res)
    
    # Calcul de similarité (exemple)
    ideal_skills = {"Python", "R", "SQL", "Machine Learning", "Big Data",
                    "Deep Learning", "Statistics", "Data Visualization",
                    "Time Series Analysis", "Cloud Computing", "Docker",
                    "Apache Spark", "Hadoop"}
    candidate_skills = {"Python", "R", "SQL", "Machine Learning"}
    similarity = compute_similarity(candidate_skills, ideal_skills)
    print("Score de similarité (exemple) :", similarity, "%")
    
if __name__ == '__main__':
    main()